# Imports

In [3]:
import pandas as pd
import datetime
from dlt.sources.helpers import requests
from bs4 import BeautifulSoup
from hashlib import md5
import re

# Teste scrappers

## Chaves na mão

### Variáveis iniciais

In [15]:
url: str = "https://www.vivareal.com.br/aluguel/parana/curitiba/?pagina=1@#onde=,Paran%C3%A1,Curitiba,,,,,city,BR%3EParana%3ENULL%3ECuritiba,,,"
#rent_html_class: str = "imoveis__Card-obm8pe-0 tNifl"

### Funções

In [17]:
# Fazer função de filtrar lista de palavras
def filter_words(words_list, desired_c=None, not_desired_c=None):
    # Retornar lista com as palavras contenham todos os desired_c e nenhum dos not_desired_c
    if desired_c and not_desired_c:
        return [word for word in words_list if any(char in word for char in desired_c) and all(char not in word for char in not_desired_c)]
    
    # Retornar lista com as palavras contenham todos os desired_c
    elif desired_c:
        return [word for word in words_list if any(char in word for char in desired_c)]

    # Retornar lista com as palavras não contenham not_desired_c
    elif not_desired_c:
        return [word for word in words_list if all(char not in word for char in not_desired_c)]

# Fazer função de pegar o possível preço do imóvel em uma lista
def get_rent_price(possible_prices: list, max_rent: float = float("inf")) -> float:
    # Fazer lista vazia para guardar os valores que realmente podem ser o preço
    new_possible_prices = []

    # Iterar as possíveis strings que contem o preço
    for price in possible_prices:
        # Se a string iterada tiver qualquer dígito numérico
        if any(l.isdigit() for l in price):
            # Mantenha apenas os numéricos ou ".", "," na string
            price = "".join([l for l in price if l.isdigit() or l in (".", ",")])

            # Em casos de preço com virgula, pegue a primeira parte do preço apenas
            price = price.split(",")[0]

            # Remova . se eles forem começo ou final de string
            price = price.strip(".")

            # Converta . em _
            price = price.replace(".", "_")
            
            # Tente
            try:
                # Conversão para floar
                price = float(price)

                # Se o preço for maior que 100_000, divida-o por esse valor
                if price > 99_999:
                    price = price / 100_000

                # Guarde ela na lista 
                new_possible_prices.append(price)

            # Em caso de erro de tipo na conversão, pule essa string
            except ValueError:
                print(f"String -> {price} não é um preço de imóvel!")

    # Filtrar alugueis abaixo de max_rent
    new_possible_prices = [price for price in new_possible_prices if price < max_rent]

    # Tente
    try:
        # Retornae o maior campo da lista
        return max(new_possible_prices)
    
    # Em caso de lista vazia
    except ValueError:
        # Retorne nuloe
        return None

# Fazer função de pegar o tamanho do imóvel
def get_rent_size(possible_sizes: list,  max_size: int = float("inf")) -> float:
    # Fazer lista vazia para guardar os valores que realmente podem ser o tamanho do imóvel
    new_possible_sizes = []

    # Iterar as possíveis strings que contem o tamanho
    for size in possible_sizes:
        # Se a string iterada tiver qualquer dígito numérico que não seja ², ³
        if any(l.isdigit() and l != "²" and l != "³" for l in size):
            # Deixe somente numeros e não deixe ², ³
            size = "".join([l for l in size if (l.isdigit() and l != "²" and l != "³") or (l == ",")])

            # Divida o tamanho na "," (se tiver) e pegue o primeiro campo
            size = size.split(",")[0]

            # Tente
            try:
                # Conversão para inteiro
                size = int(size)
            
                # Guarde ela na lista
                new_possible_sizes.append(size)

            # Em caso de erro de tipo na conversão, pule essa string
            except ValueError:
                print(f"String -> {size} não é um tamanho de imóvel!")

    # Filtrar tamanhnos abaixo de max_size
    new_possible_sizes = [size for size in new_possible_sizes if size < max_size]

    # Tente retornar o primeiro indice
    try:
        # Retorne o primeiro campo da lista
        return new_possible_sizes[0]
    
    # Se não houver itens o suficiente
    except IndexError:
        # Retorne Nuloe
        return None

# Fazer função de pegar o endereço do imóvel
def get_rent_adress(rent_splited_words: list) -> str:
    # Importar objetos   
    from rent_warehouse_mania_pipeline_objects import street_synonyms, city_names
    
    # Faça uma lista vazia para guardar os possíveis indexes
    possible_rua_index = []
    possible_city_index = []

    # Iterar todas as palavras buscando os possíveis indices da rua
    for iword, word in enumerate(rent_splited_words):
        # Se a palavra estiver na lista de sinonimos para rua, guarde esse index
        if any(i in word.lower() for i in street_synonyms):
            possible_rua_index.append(iword)

        # Se a palavra estiver na lista de cidades, guarde esse index
        if any(i in word.lower() for i in city_names):
            possible_city_index.append(iword)

    # Se ambas as listas tiverem idnex
    if len(possible_rua_index) != 0 and len(possible_city_index) != 0:
        # Retornar o join de todas as palavras na string, escolhendo o min(possible_rua_index) até o max(possible_city_index)
        return " ".join(rent_splited_words[min(possible_rua_index):max(possible_city_index)])
    
    # Do contrário
    else:
        # Retorne um join geral
        return " ".join(rent_splited_words)


### Scrapper

In [18]:
# Pegar a response
response = requests.get(url)

# Se o status vier 200, prossiga
if response.status_code == 200:
    # Pegar sopa de letras com o BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Pegar todas as divs com a classe rent_html_class
    imoveis = [imovel.text for imovel in soup.find_all("article", class_="property-card__container js-property-card")]

    di = {
        "id": [],
        "preco": [],
        "tamanho": [],
        "endereco": [],
    }
    
    # Iterar todos os imóveis
    for imovel in imoveis:
        # Splitar palavras do card do imovel
        imovel_words = imovel.split()

        # Pegar campo de preço do imovel
        preco = get_rent_price(filter_words(imovel_words, desired_c=[".", "$"], not_desired_c=["²", "³"]), max_rent=50_000)

        # Pegar campo de tamanho
        tamanho = get_rent_size(filter_words(imovel_words, desired_c=["²", "³"], not_desired_c="$"), max_size=5_000)

        # Pegar campo de endereço
        endereco = get_rent_adress(imovel_words)

        # Gerar id com hash md5
        rent_id = md5(endereco.encode("utf-8")).hexdigest()

        # apendaar na lista
        di["id"].append(rent_id)
        di["endereco"].append(endereco)
        di["preco"].append(preco)
        di["tamanho"].append(tamanho)

In [7]:
requests.get("https://www.vivareal.com.br/aluguel/parana/curitiba/?pagina=99999999999999999999999999999999999999999999999#onde=,Paran%C3%A1,Curitiba,,,,,city,BR%3EParana%3ENULL%3ECuritiba,,,")

<Response [200]>

In [23]:
imovel_words

['Indisponível',
 'Visualizado',
 '+',
 '42',
 'fotos',
 'Apartamento',
 'com',
 '3',
 'Quartos',
 'para',
 'Aluguel,',
 '104m²',
 'Cristo',
 'Rei,',
 'Curitiba',
 '-',
 'PR',
 'ver',
 'mapa',
 '104',
 'm²',
 '3',
 'Quartos',
 '2',
 'Banheiros',
 '2',
 'Vagas',
 'R$',
 '2.500',
 '/Mês',
 'Condomínio:',
 'R$',
 '700',
 'Telefone',
 'Mensagem',
 'Whatsapp']

In [22]:
imoveis

['     Indisponível Visualizado                  + 12 fotos                       Apartamento com 2 Quartos para Aluguel, 68m²    Rua Domingos Gabardo, 117 - Capão Raso, Curitiba - PR  ver mapa       68   m²     2   Quartos     2   Banheiros     1   Vaga      Mobiliado   Churrasqueira   Elevador   Varanda   Academia  ...       R$ 2.600 /Mês     Condomínio:   R$ 550         Telefone  Mensagem  Whatsapp     ',
 '     Indisponível Visualizado                  + 12 fotos                       Apartamento com 2 Quartos para Aluguel, 72m²    Rua Urbano Lopes, 402 - Cristo Rei, Curitiba - PR  ver mapa       72   m²     2   Quartos     1   Banheiro     1   Vaga      Condomínio fechado   Portão eletrônico   Portaria 24h        R$ 2.300 /Mês     Condomínio:   R$ 570         Telefone  Mensagem      ',
 '     Indisponível Visualizado                  + 6 fotos                       Apartamento com 3 Quartos para Aluguel, 45m²    Rua Padre José Lopacinski, 580 - Cidade Industrial, Curitiba - PR  ve